The maps generated with these data are really cool. But since I'm pretty new to coding I will try and visualize the data in another way. 

In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt

from pandas import set_option
set_option("display.max_rows", None)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
df = pd.read_csv("../input/database.csv")

In [26]:
df.head()

I will try to help visualize dominant rock type associated with different tectonic settings. To do this, I don't need all these categories, so removing some.

In [27]:
df = df.drop(['Number', 'Activity Evidence', 'Last Known Eruption', 'Latitude', 'Longitude', 'Elevation (Meters)'], axis=1)

Will check to see what this condensed dataset looks like.

In [28]:
df.info()

Missing some Tectonic setting data and Dominant rock type data. Will explore this further and try to clean it up.

In [29]:
df['Tectonic Setting'] = df['Tectonic Setting'].replace("Unknown", np.nan)

In [30]:
mask1 = df['Tectonic Setting'].notnull()
df = df[mask1]
df.info()

Getting better, but a lot more to do. If we look closer at the data, there are inconsistencies. As you can see below there are some duplicates. Is it a stratovolcano or stratavolcano (es)? This will create problems and adds a dimension we don't need, so I will attempt to fix it. For the purpose of this exercise, I'm going to assume all groups with (es) should be included with the others. Also, there are four "Type" = "stratovolcano?". I'll be consistent and again include these all as a part of the "Stratovolcano" "Type". You could probably do this other ways and maybe a volcanologist would object, but I'm just trying to learn code.

In [31]:
df['Type'].value_counts()

In [32]:
df['Type'] = df['Type'].str.replace(r"\(.*\)", "")

In [33]:
df['Type'] = df['Type'].str.replace("?", "")

In [34]:
df['Type'] = df['Type'].replace("Unknown", np.nan)

In [35]:
mask2 = df['Type'].notnull()
df = df[mask2]
df.info()

Similar issues with the Dominant Rock Type category

In [36]:
df['Dominant Rock Type'] = df['Dominant Rock Type'].str.replace("No Data", "")

In [37]:
df['Dominant Rock Type'] = df['Dominant Rock Type'].replace("", np.nan)

In [38]:
mask3 = df['Dominant Rock Type'].notnull()
df = df[mask3]
df.head(5)

In [39]:
df['Tectonic Setting'].value_counts()

In [40]:
df['Dominant Rock Type'].value_counts()

These data are skewed for sure. But again, just messing around with learning code. These data are also a little unique in that we are looking at classification of rock types and volcanoes, not values or numbers. Which might mean I picked a bad dataset to try and use python/numpy/pandas on! But its too late! I'll move on and try and categorize and assign some numbers.

In [41]:
for i, value in enumerate(df['Dominant Rock Type'].sort_values().unique()):
    df.loc[df['Dominant Rock Type'] == value, 'DRT'] = i

With this I'm creating a new object "DRT" which assigns a number to each unique category of Dominant Rock Type. Note the sorting to create alphabetical instead of by number, this helps later.

In [42]:
df.head()

To display things in a way thats more helpful, I'll assign some colors and provide some acronyms instead of the insanely wordy (but descriptive!) Dominant Rock Types.

In [43]:
#This is the sorted unique order for each new ocurrence of Dominant Rock Type.
DRT_names = ['A_BA', 'B_PB', 'Dac', 'Fd', 'P_TP', 'Ph', 'Rhy', 'TaBTa', 'TbTB', 'TTd']
DRT_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00', '#1B4F72','#2E86C1', '#AED6F1', '#A569BD', '#196F3D', 'black']

tec_set = df['Tectonic Setting'].values
DRT = df['DRT'].values

In [44]:
mpl.rcParams.update({'font.size': 10})
mpl.rcParams['figure.figsize'] = (15.0, 15.0)
inline_rc = dict(mpl.rcParams)

for tectonic, tec in enumerate(np.unique(tec_set)):
    ax = plt.subplot(4, 3, tectonic+1)
    hist = np.histogram(DRT[tec_set == tec], bins=np.arange(len(DRT_names)+1))
    plt.bar(np.arange(len(hist[0])), hist[0], color=DRT_colors, align='center')
    ax.set_xticks(np.arange(len(hist[0])))
    ax.set_xticklabels(DRT_names)
    ax.set_title(tec)

So here we have a display of the different tectonic settings and the Dominant Rock Type. Just another way to view the data instead of the maps. Which are very cool!
I learned most of this from this contest here. https://github.com/seg/2016-ml-contest